In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import os

def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)
    return fd

def detect_board(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        if len(approx) == 4:  # Phát hiện hình vuông hoặc chữ nhật
            return approx
    
    return None

def extract_board_matrix(image):
    board_corners = detect_board(image)
    if board_corners is None:
        return None, None
    
    # Sắp xếp các góc theo thứ tự trái trên, phải trên, phải dưới, trái dưới
    board_corners = board_corners.reshape(4, 2)
    board_corners = sorted(board_corners, key=lambda x: (x[1], x[0]))
    
    # Xác định kích thước bàn cờ (ví dụ 5x5, 10x10...)
    size = 10  # Mặc định 10x10, có thể thay đổi dựa trên dữ liệu
    
    # Biến đổi phối cảnh để chuẩn hóa bàn cờ
    src_pts = np.float32(board_corners)
    dst_pts = np.float32([[0, 0], [size * 50, 0], [size * 50, size * 50], [0, size * 50]])
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, matrix, (size * 50, size * 50))
    
    # Chia nhỏ bàn cờ thành các ô vuông
    cell_size = 50
    board_matrix = np.zeros((size, size), dtype=int)
    
    for i in range(size):
        for j in range(size):
            cell = warped[i * cell_size:(i + 1) * cell_size, j * cell_size:(j + 1) * cell_size]
            gray_cell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
            _, binary_cell = cv2.threshold(gray_cell, 128, 255, cv2.THRESH_BINARY)
            
            # Xác định X hoặc O (giả định đơn giản: X có nhiều cạnh hơn O)
            edges = cv2.Canny(binary_cell, 50, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if len(contours) > 5:  # X có nhiều đường biên hơn O
                board_matrix[i, j] = 1  # X
            elif len(contours) > 0:
                board_matrix[i, j] = -1  # O
    
    return board_matrix, warped

def recognize_caro_board():
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        board_matrix, processed_board = extract_board_matrix(frame)
        
        if board_matrix is not None:
            print("Bàn cờ nhận diện:")
            print(board_matrix)
            cv2.imshow("Processed Board", processed_board)
        
        cv2.imshow("Camera", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    recognize_caro_board() 

In [9]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib
import os

def extract_hog_features(image):
    if len(image.shape) == 3:  # Kiểm tra nếu ảnh có 3 kênh màu (BGR)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image  # Ảnh đã là grayscale thì giữ nguyên
    
    fd = hog(gray, orientations=9, pixels_per_cell=(8, 8),
             cells_per_block=(2, 2), visualize=False, feature_vector=True)
    
    fixed_hog_size = 84  # Đảm bảo số đặc trưng trùng khớp với dữ liệu huấn luyện
    if len(fd) < fixed_hog_size:
        fd = np.pad(fd, (0, fixed_hog_size - len(fd)), mode='constant')
    else:
        fd = fd[:fixed_hog_size]
    
    return fd

def detect_board(image, model):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = extract_hog_features(image)
    prediction = model.predict([features])
    
    if prediction[0] == 1:  # Giả định rằng nhãn 1 tương ứng với bàn cờ được phát hiện
        return cv2.findContours(cv2.Canny(gray, 50, 150), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    return None

def get_board_position(image, board_corners):
    if board_corners is None:
        return None
    
    board_corners = np.array(board_corners, dtype=np.int32)
    x, y, w, h = cv2.boundingRect(board_corners)
    return (x, y, w, h)

def extract_board_matrix(image, model):
    board_corners = detect_board(image, model)
    if board_corners is None:
        return None, None, 0, None
    
    size = 10  
    
    board_corners = np.array(board_corners).reshape(-1, 2)
    src_pts = np.float32(board_corners[:4])
    dst_pts = np.float32([[0, 0], [size * 50, 0], [size * 50, size * 50], [0, size * 50]])
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, matrix, (size * 50, size * 50))
    
    cell_size = 50
    board_matrix = np.zeros((size, size), dtype=int)
    
    for i in range(size):
        for j in range(size):
            cell = warped[i * cell_size:(i + 1) * cell_size, j * cell_size:(j + 1) * cell_size]
            features = extract_hog_features(cell)
            prediction = model.predict([features])
            board_matrix[i, j] = prediction[0]  
    
    board_position = get_board_position(image, board_corners)
    return board_matrix, warped, size, board_position

def recognize_caro_board():
    model = joblib.load("svm_caro_model.pkl")
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        board_matrix, processed_board, board_size, board_position = extract_board_matrix(frame, model)
        
        if board_matrix is not None:
            print(f"Bàn cờ nhận diện ({board_size}x{board_size}):")
            print(board_matrix)
            print(f"Vị trí bàn cờ trên khung hình: {board_position}")
            cv2.imshow("Processed Board", processed_board)
        
        cv2.imshow("Camera", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    recognize_caro_board()
